# AutoEncoder (1) AutoEncoder by Multi-Layer Perceptron

- [L1aoXingyu@github]()の，[Linearのみで構成されたAutoEncoderの実装](https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py)に基づいている．  
- 一部，自前環境のPyTorchで動作しない部分を修正しつつ，結果の可視化等の追加している．  

In [1]:
import torch
import os

## create folder in advance
folder = './data/mlp_img'
if not os.path.isdir(folder):
    os.mkdir(folder)

## set folder in advance
model_path = './data/mlp_autoencoder.pth'

## set some constants for learning
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

## (1) Prepare dataset: MNIST hand-written digits



In [2]:
from torchvision.datasets import MNIST
from torchvision import transforms

## image to tensor
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) ## convert range to [-1.0:1.0]
])

## tensor to image
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

## dataset with conversion
dataset_train = MNIST('./data', train=True, download=True, transform=img_transform)

In [3]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

## (2) Prepare model (i) autoencoder

Simple AutoEncoder constructed by multi-layer perceptron, which is using `nn.Linear`.  

In [4]:
from torch import nn

## definition of network
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128), ## image (28x28) -> (128)
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
## instantiate model
model = autoencoder().cuda() ## send to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

## (3) Training model

Once you already have the trained result, you may skip this part...

In [6]:
from torch.autograd import Variable
from torchvision.utils import save_image

## training
model.train()

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda() ## send to GPU
        
        ## feed-forward
        output = model(img)
        loss = criterion(output, img)
        
        ## backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # ===================log========================
    print('epoch [{0}/{1}], loss:{2:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, '{0}/image_{1}.png'.format(folder, epoch))

epoch [1/100], loss:0.2033
epoch [2/100], loss:0.1696
epoch [3/100], loss:0.1617
epoch [4/100], loss:0.1533
epoch [5/100], loss:0.1575
epoch [6/100], loss:0.1548
epoch [7/100], loss:0.1456
epoch [8/100], loss:0.1460
epoch [9/100], loss:0.1427
epoch [10/100], loss:0.1431
epoch [11/100], loss:0.1383
epoch [12/100], loss:0.1422
epoch [13/100], loss:0.1381
epoch [14/100], loss:0.1400
epoch [15/100], loss:0.1434
epoch [16/100], loss:0.1407
epoch [17/100], loss:0.1375
epoch [18/100], loss:0.1272
epoch [19/100], loss:0.1340
epoch [20/100], loss:0.1337
epoch [21/100], loss:0.1263
epoch [22/100], loss:0.1243
epoch [23/100], loss:0.1450
epoch [24/100], loss:0.1234
epoch [25/100], loss:0.1247
epoch [26/100], loss:0.1314
epoch [27/100], loss:0.1267
epoch [28/100], loss:0.1390
epoch [29/100], loss:0.1373
epoch [30/100], loss:0.1304
epoch [31/100], loss:0.1410
epoch [32/100], loss:0.1268
epoch [33/100], loss:0.1306
epoch [34/100], loss:0.1304
epoch [35/100], loss:0.1258
epoch [36/100], loss:0.1417
e

In [7]:
## save trained model
torch.save(model.state_dict(), model_path)

## (4) Testing model

In [8]:
## load trained model
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint)
model.eval() ## switch to "evaluate" mode

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [9]:
from torch.utils.data import DataLoader

## load test data & loader
dataset_test = MNIST('./data', train=False, download=True, transform=img_transform)
testloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [10]:
from torch.autograd import Variable

for test in testloader:
    img, _ = test
    save_image(img, '{0}/test_input.png'.format(folder))
    img = img.view(img.size(0), -1)
    img = Variable(img).cuda() ## send to GPU
    
    ## feed-forward
    output = model(img)
    
    pic = to_img(output.cpu().data)
    save_image(pic, '{0}/test_output.png'.format(folder))
    
    break ## generates one batch

(end)